In [2]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
import numpy as np
from torchvision import transforms
import cv2
import glob
import random

In [4]:
labels = dict(((str(i),str(train_dataset.targets[i])) for i in range(50000)))
print('# 训练样本 :', len(labels))
print('# 类别 :', len(set(labels.values())))

# 训练样本 : 50000
# 类别 : 100


In [5]:
def mixup(png1,png2,mixup_png):
    img1 = cv2.imread(png1)
    img1 = cv2.resize(img1, (32, 32))

    img2 = cv2.imread(png2)
    img2 = cv2.resize(img2, (32, 32))

    alpha = 1.0
    lam = np.random.beta(alpha, alpha)
    mixed_img = lam * img1 + (1 - lam) * img2

    cv2.imwrite(mixup_png, mixed_img)

In [6]:
path = '/Users/lizijian/data/cifar-100-python/mixup_dataset/train/*'
a = 0
for i in random.sample(range(0,50000),10000):
    a+=1
    j = np.random.randint(50000)
    file_abs_1 = glob.glob(path)[i]
    file_abs_2 = glob.glob(path)[j]
    mixup(file_abs_1,file_abs_2,file_abs_2)
    print(a,file_abs_1)

1 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/43156_18+caterpillar_s_000659.png
2 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/43586_18+caterpillar_s_000454.png
3 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10444_1+cichlid_s_000827.png
4 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/23596_51+mushroom_s_000234.png
5 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/20411_61+plate_s_002077.png
6 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/4607_76+skyscraper_s_000553.png
7 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/25485_75+mephitis_mephitis_s_000457.png
8 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/19438_51+morchella_esculenta_s_000702.png
9 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/7914_29+stegosaurus_s_001427.png
10 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/14319_50+fieldmouse_s_000148.png
11 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/21

89 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/27421_37+house_s_000717.png
90 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/12078_4+beaver_s_001884.png
91 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/13559_24+oriental_cockroach_s_001240.png
92 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/4186_70+rosebush_s_001988.png
93 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/49129_18+caterpillar_s_001468.png
94 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/12261_84+table_s_002956.png
95 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/47712_10+bowl_s_002343.png
96 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/46262_23+cirrostratus_s_000813.png
97 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/8194_74+sorex_araneus_s_000171.png
98 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/40369_81+streetcar_s_000874.png
99 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/15840_93+tu

177 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/7320_59+pine_tree_s_001960.png
178 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10409_93+sea_turtle_s_000975.png
179 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/14648_81+streetcar_s_001404.png
180 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/2509_12+footbridge_s_000594.png
181 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/6654_41+lawn_mower_s_001267.png
182 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/39759_16+tin_can_s_001597.png
183 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/15928_13+trolleybus_s_000238.png
184 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/12841_36+golden_hamster_s_000505.png
185 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/23744_48+motorcycle_s_000617.png
186 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/41526_29+armored_dinosaur_s_000039.png
187 /Users/lizijian/data/cifar-100-python/mixup_

265 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/19253_41+lawn_mower_s_000859.png
266 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/41846_79+argiope_aurantia_s_000380.png
267 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/40633_98+white_woman_s_000424.png
268 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/23700_26+crab_s_000639.png
269 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/32239_49+mount_s_001010.png
270 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/29093_85+armoured_combat_vehicle_s_000260.png
271 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/13335_34+fox_s_001814.png
272 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/30013_31+elephant_s_000929.png
273 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/48879_54+butterfly_orchid_s_000403.png
274 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/31176_92+tulip_s_002210.png
275 /Users/lizijian/data/cifar-100-python/mixup_d

353 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/6706_80+squirrel_s_001859.png
354 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/19798_92+lady_tulip_s_000749.png
355 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/24939_45+american_lobster_s_000762.png
356 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/16095_52+quercus_bicolor_s_000086.png
357 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/13922_50+harvest_mouse_s_000925.png
358 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10056_96+salix_vitellina_s_000033.png
359 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/34236_98+black_woman_s_000258.png
360 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/28032_26+stone_crab_s_001585.png
361 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/24378_40+discharge_lamp_s_000569.png
362 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10037_0+pearmain_s_000365.png
363 /Users/lizijian/data/cifar-1

...

9577 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/5081_38+wallaby_s_000196.png
9578 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/2095_62+poppy_s_001039.png
9579 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/39835_36+syrian_hamster_s_000533.png
9580 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/976_54+orchid_s_001119.png
9581 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/3046_19+cows_s_002039.png
9582 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/25712_71+sea_s_000461.png
9583 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/31739_77+garden_snail_s_001575.png
9584 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/6832_8+cycle_s_000583.png
9585 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/4962_99+nematode_s_001791.png
9586 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/36473_45+northern_lobster_s_000010.png
9587 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/37399

9665 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/47715_78+coluber_hippocrepis_s_000036.png
9666 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/42286_97+wolf_pup_s_001427.png
9667 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/34588_43+lion_cub_s_000600.png
9668 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10113_64+possum_s_001409.png
9669 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/38593_39+computer_keyboard_s_002070.png
9670 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/33115_11+male_child_s_000956.png
9671 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/14746_37+boardinghouse_s_000336.png
9672 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/11892_86+desk_phone_s_000633.png
9673 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/3691_33+wood_s_001655.png
9674 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/35116_75+striped_skunk_s_000038.png
9675 /Users/lizijian/data/cifar-1

9753 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/5454_82+sunflower_s_000037.png
9754 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/1360_26+stone_crab_s_000200.png
9755 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/1055_27+crocodile_s_000036.png
9756 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/4292_34+red_fox_s_001682.png
9757 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/24973_96+salix_purpurea_s_000383.png
9758 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/31483_28+goblet_s_001547.png
9759 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/40788_10+bowl_s_000302.png
9760 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/28602_63+hedgehog_s_000514.png
9761 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/48810_86+telephone_s_000743.png
9762 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/43193_97+timber_wolf_s_000290.png
9763 /Users/lizijian/data/cifar-100-python/mixup_dataset/

9841 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/19455_42+leopard_s_002236.png
9842 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/14217_72+seal_s_000665.png
9843 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/22715_42+leopard_s_001071.png
9844 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/24169_60+field_s_000015.png
9845 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/24517_66+raccoon_s_000317.png
9846 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/39804_39+computer_keyboard_s_001317.png
9847 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/48235_96+willow_tree_s_002446.png
9848 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/21640_92+tulip_s_000011.png
9849 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/44289_13+minibus_s_000840.png
9850 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/39837_14+mourning_cloak_s_000776.png
9851 /Users/lizijian/data/cifar-100-python/mixup_dataset

9929 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/34689_37+boardinghouse_s_001363.png
9930 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/8211_43+panthera_leo_s_001949.png
9931 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/28758_60+serengeti_s_001142.png
9932 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/44981_45+northern_lobster_s_000718.png
9933 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/19442_66+raccoon_s_000373.png
9934 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/28904_52+quercus_agrifolia_s_000008.png
9935 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/10836_14+butterfly_s_000360.png
9936 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/43050_60+field_s_002834.png
9937 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/15460_83+sweet_pepper_s_001025.png
9938 /Users/lizijian/data/cifar-100-python/mixup_dataset/train/38887_98+white_woman_s_000052.png
9939 /Users/lizijian/data/cifa